# DC$\pi$BO - Stationary DAG and SCM

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../src/")
sys.path.append("../..")

from src.examples.example_setups import setup_stat_scm
from src.utils.sem_utils.toy_sems import StationaryDependentSEM as StatSEM
from src.utils.sem_utils.sem_estimate import build_sem_hat

from src.experimental.experiments import run_methods_replicates
from src.experimental.analyse_results import get_relevant_results, elaborate, gap_metric_standard, get_common_initial_values, get_converge_trial
from src.utils.plotting import plot_expected_opt_curve_paper

from src.methods.bo import BO
from src.methods.dcbo import DCBO
from src.methods.pibo import PIBO
from src.methods.dcpibo import DCPIBO

from matplotlib.pyplot import rc
import numpy as np
from scipy import stats
import random

random.seed(42)

## Utility Functions

In [2]:
def power_list(input_list, exponent):
    '''Raises each element of the given input_list to the desired exponent'''
    
    return_list = []
    
    for element in input_list:
        if element >= 0:
            raised_element = element**exponent
        else:
            raised_element = -(abs(element)**exponent)
        return_list.append(raised_element)
        
    return return_list

In [3]:
def compute_statistics(y_function, standard_deviation):
    '''Computes all the necessary lists associated to the Normal Distribution'''
    
    return_variates = []
    return_amps = []
    return_pdfs = []
    
    for mean in y_function:
        return_variates.append(stats.norm.rvs(mean, standard_deviation, 10))
        
        amp = np.linspace(mean-5*standard_deviation, mean+5*standard_deviation, 10)
        return_amps.append(amp)
        
        return_pdfs.append(stats.norm.pdf(amp, mean, standard_deviation))
    
    return return_variates, return_amps, return_pdfs

In [4]:
def normalise_pdfs(pdfs_list):
    '''Normalises the PDFs between 0 and 1'''
    
    return_normalised_pdfs_list = []
    
    for pdf_list in pdfs_list:
        temp_list = []
        
        pdf_min = min(pdf_list)
        pdf_max = max(pdf_list)
        
        for pdf_value in pdf_list:
            temp_list.append(round((pdf_value-pdf_min)/(pdf_max-pdf_min),2))
        
        return_normalised_pdfs_list.append(temp_list)
        
    return return_normalised_pdfs_list

In [5]:
def regret_priors(start_prior):
    '''Adds regret'''
    
    return_raised_priors = []
    
    for i in range(1,N+1):
        temp = []
        gamma = beta/i
        
        for p_list in start_prior:
            temp.append(power_list(p_list, gamma))
            
        return_raised_priors.append(temp)
    
    return return_raised_priors

In [6]:
def predict_optima_regret(iterations, time_steps, regret_priors, normalised_pdfs):
    '''Computes the prediction for each time-step and each iteration, according to the effect of gamma (regret)'''
    
    return_predictions = []
    
    for iteration in range(iterations):
        temp = []
        
        for time_step in range(time_steps):
            if(min(regret_priors[iteration][time_step])+max(regret_priors[iteration][time_step]))<0:
                optimum = min(np.multiply(regret_priors[iteration][time_step], normalised_pdfs[time_step]))
            else:
                optimum = max(np.multiply(regret_priors[iteration][time_step], normalised_pdfs[time_step]))
                              
            temp.append(optimum)
                              
        return_predictions.append(temp)
                              
    return return_predictions

## Constants and such

In [7]:
T = 5
t_axis = np.linspace(1,T,T,dtype=int)
N = 20
std = 0.1
beta = 1

## Optima location

### Good prior

In [8]:
a_param = -2.15
b_param = 1
y = a_param*t_axis+b_param # the line on which I assume the optima would be

### Bad prior

In [9]:
bad_a_param = 6
bad_y_param = 4
bad_y = bad_a_param*t_axis+bad_y_param

## Setup

### Good prior:

In [10]:
all_variates, all_amps, all_pdfs = compute_statistics(y, std)
all_variates_regret = regret_priors(all_variates)

all_pdfs_normalised = normalise_pdfs(all_pdfs)

predicted_optima_regret = predict_optima_regret(N, T, all_variates_regret, all_pdfs_normalised) # this are all the 𝜋s

### Bad prior

In [11]:
bad_all_variates, bad_all_amps, bad_all_pdfs = compute_statistics(bad_y, std)
bad_all_variates_regret = regret_priors(bad_all_variates)

bad_all_pdfs_normalised = normalise_pdfs(bad_all_pdfs)

bad_predicted_optima_regret = predict_optima_regret(N, T, bad_all_variates_regret, bad_all_pdfs_normalised) # this are all the 𝜋s

### Model:

In [12]:
init_sem, sem, dag_view, G, exploration_sets, intervention_domain, true_objective_values, optimal_interventions, all_causal_effects  = setup_stat_scm(T=T)

## Models' executions

In [13]:
R = 3

### Good prior

In [14]:
results = run_methods_replicates(G=G, 
                                sem=StatSEM, 
                                make_sem_estimator=build_sem_hat, 
                                base_target_variable='Y',
                                intervention_domain = intervention_domain, 
                                methods_list = ['BO', 'PIBO', 'DCBO', 'DCPIBO'],
                                obs_samples = None,
                                exploration_sets = exploration_sets,
                                priors_regret = predicted_optima_regret,
                                total_timesteps = T,
                                number_of_trials = N, 
                                reps = R, # Number of replicates (how many times we run each method)
                                n_restart = 1,
                                save_data = False,
                                n_obs = 5, # The method samples 5 time-series for each replicate
                                num_anchor_points = 100,
                                sample_anchor_points = True,
                                controlled_experiment=False)

Experiment count:   0%|                                                                                                                | 0/3 [00:00<?, ?it/s]


	>>>BO




Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  2.86it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:01,  2.93it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:01<00:00,  2.87it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  2.82it/

20

	>>>PIBO




Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  3.27it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:00,  3.06it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:00<00:00,  3.04it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  2.93it/

...


Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  3.20it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:00,  3.12it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:00<00:00,  3.00it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  2.86it/


	>>>DCBO



 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance

Time index:  20%|████████


	>>>DCPIBO



 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance

Time index:  20%|████████

### Bad prior

In [15]:
bad_results = run_methods_replicates(G=G, 
                                sem=StatSEM, 
                                make_sem_estimator=build_sem_hat, 
                                base_target_variable='Y',
                                intervention_domain = intervention_domain, 
                                methods_list = ['BO', 'PIBO', 'DCBO', 'DCPIBO'],
                                obs_samples = None,
                                exploration_sets = exploration_sets,
                                priors_regret = bad_predicted_optima_regret,
                                total_timesteps = T,
                                number_of_trials = N, 
                                reps = R, # Number of replicates (how many times we run each method)
                                n_restart = 1,
                                save_data = False,
                                n_obs = 5, # The method samples 5 time-series for each replicate
                                num_anchor_points = 100,
                                sample_anchor_points = True,
                                controlled_experiment=False)

Experiment count:   0%|                                                                                                                | 0/3 [00:00<?, ?it/s]


	>>>BO




Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  3.42it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:00,  3.15it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:00<00:00,  3.16it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  2.90it/

20

	>>>PIBO




Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  3.27it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:00,  3.14it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:00<00:00,  3.11it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  3.06it/

...


Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  20%|██████████████████████                                                                                        | 1/5 [00:00<00:01,  3.20it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  40%|████████████████████████████████████████████                                                                  | 2/5 [00:00<00:00,  3.13it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  60%|██████████████████████████████████████████████████████████████████                                            | 3/5 [00:00<00:00,  2.97it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  80%|████████████████████████████████████████████████████████████████████████████████████████                      | 4/5 [00:01<00:00,  2.93it/


	>>>DCBO



 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance

Time index:  20%|████████


	>>>DCPIBO



 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

Time index:   0%|                                                                                                                      | 0/5 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance

Time index:  20%|████████

## Plotting

In [16]:
plot_params = {
    "linewidth": 3,
    "linewidth_opt": 4,
    "alpha": 0.1,
    "xlim_max": N,
    "ncols": 5,
    "loc_legend": "lower right",
    "size_ticks": 20,
    "size_labels": 20,
    "xlabel": r'$\texttt{cost}(\mathbf{X}_{s,t}, \mathbf{x}_{s,t})$',
    "labels": {'DCPIBO': 'DCPIBO', 'PIBO': 'PIBO', 'DCBO': 'DCBO', 'BO': 'BO', 'True': r'$\mathbb{E} \left [Y_t \mid \textrm{do}(\mathbf{X}_{s,t}^\star = \mathbf{x}_{s,t}^\star) \right]$'},
    "colors": {'DCPIBO': 'blue', 'PIBO': 'green', 'DCBO': 'orange', 'BO': 'red', 'True': 'black'},
    "line_styles": {'DCPIBO': 'dashdot', 'PIBO': '--', 'DCBO': '-', 'BO': '-', 'True': ':'},
    "width":10
}
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amssymb}')
rc('font', family='serif')
rc('font', size=20)

### Good prior

In [17]:
data = get_relevant_results(results=results,replicates=R)
exp_optimal_outcome_values_during_trials, exp_per_trial_cost = elaborate(number_of_interventions=None, 
                                                                         n_replicates=R, 
                                                                         data=data, 
                                                                         best_objective_values=true_objective_values, 
                                                                         T=T)

In [18]:
# # Each frame corresponds to one time-slice.
# plot_expected_opt_curve_paper(T,
#     true_objective_values,
#     exp_per_trial_cost,
#     exp_optimal_outcome_values_during_trials,
#     plot_params,  
#     fig_size = (15,5))

### Bad prior

In [19]:
bad_data = get_relevant_results(results=bad_results,replicates=R)
bad_exp_optimal_outcome_values_during_trials, bad_exp_per_trial_cost = elaborate(number_of_interventions=None, 
                                                                         n_replicates=R, 
                                                                         data=bad_data, 
                                                                         best_objective_values=true_objective_values, 
                                                                         T=T)

In [20]:
# # Each frame corresponds to one time-slice.
# plot_expected_opt_curve_paper(T,
#     true_objective_values,
#     bad_exp_per_trial_cost,
#     bad_exp_optimal_outcome_values_during_trials,
#     plot_params,  
#     fig_size = (15,5))

## Gap Metric

### Good Gap Metric

In [21]:
t_i_l = get_common_initial_values(T=T,
                                  data=data,
                                  n_replicates=R)

w_c_d = get_converge_trial(best_objective_values=true_objective_values,
                          exp_optimal_outcome_values_during_trials=exp_optimal_outcome_values_during_trials,
                          n_trials=N,
                          T=T,
                          n_decimal=1)

summary = gap_metric_standard(T=T,
                             data=data,
                             best_objective_values=true_objective_values,
                             total_initial_list = t_i_l,
                             n_replicates=R,
                             n_trials=N,
                             where_converge_dict=w_c_d)

summary

{'BO': [[0.6147051295393695, 0.0001668447797210501],
  [0.8184780012356269, 0.0013476892189889867],
  [0.5094658920224168, 0.001826062165340366],
  [0.9159432362994225, 0.0037234673760761982],
  [0.5057011467792145, 0.0005488839717574976]],
 'PIBO': [[0.5082612480673269, 0.0007054481876730316],
  [0.7632033359149798, 0.0022568849407810575],
  [0.5072685725793802, 0.001365955778166275],
  [0.5728577132689466, 0.01163062221380183],
  [0.49938182710336115, 0.009363718593854016]],
 'DCBO': [[0.5116224416441305, 0.00036112313242224903],
  [0.47015323765604106, 0.02792859104344767],
  [0.48439464447906255, 0.012207668369226259],
  [0.4466979606400809, 0.0411894210098653],
  [0.4730956535833557, 0.01806650200745402]],
 'DCPIBO': [[0.5381345443822341, 0.00020569167676105695],
  [0.4619736933211646, 0.03577715505011915],
  [0.47872047177318916, 0.02378922492076734],
  [0.41940572688832334, 0.05626856840767716],
  [0.45833828324905457, 0.019390098538837476]]}

### Bad Gap Metric

In [22]:
bad_t_i_l = get_common_initial_values(T=T,
                                  data=data,
                                  n_replicates=R)

bad_w_c_d = get_converge_trial(best_objective_values=true_objective_values,
                          exp_optimal_outcome_values_during_trials=bad_exp_optimal_outcome_values_during_trials,
                          n_trials=N,
                          T=T,
                          n_decimal=1)

bad_summary = gap_metric_standard(T=T,
                             data=data,
                             best_objective_values=true_objective_values,
                             total_initial_list = bad_t_i_l,
                             n_replicates=R,
                             n_trials=N,
                             where_converge_dict=bad_w_c_d)

bad_summary

{'BO': [[0.5121410269752671, 0.0001668447797210501],
  [0.6902728730304987, 0.0013476892189889867],
  [0.5094658920224168, 0.001826062165340366],
  [0.5056868260430122, 0.003723467376076163],
  [0.5057011467792145, 0.0005488839717574976]],
 'PIBO': [[0.5082612480673269, 0.0007054481876730316],
  [0.7888443615560056, 0.002256884940781007],
  [0.5072685725793802, 0.001365955778166275],
  [0.7010628414740748, 0.01163062221380178],
  [0.49938182710336115, 0.009363718593854016]],
 'DCBO': [[0.5116224416441305, 0.00036112313242224903],
  [0.47015323765604106, 0.02792859104344767],
  [0.48439464447906255, 0.012207668369226259],
  [0.4466979606400809, 0.0411894210098653],
  [0.4730956535833557, 0.01806650200745402]],
 'DCPIBO': [[0.6150576213053109, 0.0002056916767610942],
  [0.4619736933211646, 0.03577715505011915],
  [0.47872047177318916, 0.02378922492076734],
  [0.41940572688832334, 0.05626856840767716],
  [0.45833828324905457, 0.019390098538837476]]}